In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
#drive.flush_and_unmount()
#!rm -rf /content/drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install tf-keras-vis
# -*- coding: utf-8 -*-
"""GradCam.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1cKjutZfO-CAwKnI4E84NAhm4seqoyGi0
"""

#!pip install tf-keras-vis

import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import time
#from Models.loss import smoothL1
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Input, InputLayer
from tensorflow.keras.layers import Activation, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2

from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam import GradcamPlusPlus

from tf_keras_vis.utils import normalize

import matplotlib.pyplot as plt



import math
import io
import os
from collections import namedtuple
import sys

import operator

import copy

#from tf_keras_vis.utils import num_of_gpus

#_, gpus = num_of_gpus()
#print('{} GPUs'.format(gpus))
!pwd
os.chdir("./drive/MyDrive/Colab Notebooks")
!pwd
!ls

print(tf.__version__)

from platform import python_version

print(python_version())

In [ ]:
import os
import keras
import argparse
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
#from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, Dense, Input, add, Activation, GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler
from keras.models import Model, load_model
from keras import optimizers, regularizers
from keras import backend as K
import pickle

def get_prediction(img, tensor = None):

    if tensor == "ALL":
        full_list = []
        for tensor in ['dense_1', 'dense_2', 'dense_3']:
            layer_name = tensor
            layer = model.get_layer(layer_name)
            func = K.function(model.input, [layer.output])
            imag = np.expand_dims(img,axis=0)
            outputs = func(imag)
           # print(outputs)
            layer_outputs_test = []
            fingerprint1 = ((outputs[0] > 0.0).astype('int'))
            layer_outputs_test = []
            layer_result = fingerprint1
            layer_outputs_test.append(layer_result[0])
            full_list.append(layer_outputs_test)

        new_list = [list(full_list[0][0])+list(full_list[1][0])+list(full_list[2][0])]
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    else:
        layer_name = tensor
        layer = model.get_layer(layer_name)
        func = K.function(model.input, [layer.output])
        imag = np.expand_dims(img,axis=0)
        outputs = func(imag)
       # print(outputs)
        fingerprint1 = ((outputs[0] > 0.0).astype('int'))
        layer_outputs_test = []
        layer_result = fingerprint1
        layer_outputs_test.append(layer_result[0])
        layer_outputs_test = np.asarray(layer_outputs_test)

        return layer_outputs_test

def get_prediction_vals(img, tensor = None):

    if tensor == "ALL":
        full_list = []
        for tensor in ['activation_1','activation_2','activation_3']:
            layer_name = tensor
            layer = model.get_layer(layer_name)
            func = K.function(model.input, [layer.output])
            imag = np.expand_dims(img,axis=0)
            outputs = func(imag)
            op = np.array(outputs)
            #print(op.shape)
            flat_op = op.flatten()
            #print(flat_op.shape)
            fingerprint1 = list(flat_op)
            #print(layer.name,fingerprint1)
            layer_outputs_test = []
            layer_result = fingerprint1
            layer_outputs_test.append(layer_result)
            full_list.append(layer_outputs_test)

        new_list = [list(full_list[0][0])+list(full_list[1][0])+list(full_list[2][0])]
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    else:
        layer_name = tensor
        layer = model.get_layer(layer_name)
        func = K.function(model.input, [layer.output])
        imag = np.expand_dims(img,axis=0)
        outputs = func(imag)
       # print(outputs)
        #fingerprint1 = ((outputs[0] > 0.0).astype('int'))
        fingerprint1 = outputs[0]
        layer_outputs_test = []
        layer_result = fingerprint1
        layer_outputs_test.append(layer_result[0])
        layer_outputs_test = np.asarray(layer_outputs_test)

        return layer_outputs_test





def fingerprint_suffix(img,ten):
    return (get_prediction(img, tensor=ten)>0.0).astype('int')

def fingerprint_suffix_vals(img,ten):
    return get_prediction_vals(img, tensor=ten)

#from tf_keras_vis.gradcam import Gradcam
#from tf_keras_vis.gradcam import GradcamPlusPlus

def model_modifier_act1_layer(current_model):
    target_layer = current_model.get_layer(name='activation_1') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_act2_layer(current_model):
    target_layer = current_model.get_layer(name='activation_2') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_act3_layer(current_model):
    target_layer = current_model.get_layer(name='activation_3') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_last_layer(current_model):
    target_layer = current_model.get_layer(name='activation_5') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer(current_model):
    target_layer = current_model.get_layer(name='dense') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer_inps(current_model):
    src_layer = current_model.get_layer(name='activation')
    target_layer = current_model.get_layer(name='dense') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=src_layer.input,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def loss_gen_sum(node_list):
    def loss(output):
        print(output.shape)
        op = np.array(output)

        #if (op.ndim > 2):
        op = op.flatten()
        output = np.empty((1,len(op)))
        print(output.shape)
        output[0] = op

        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss


def loss_gen_sum1(node_list):
    def loss(output):

        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss

def loss_gen_sep(node):
    def loss(output):
        loss_val = output[0][node]
        return loss_val
    return loss

#import matplotlib.pyplot as plt
#from keras.models import load_model
print('Loading the model:')
model=load_model('cifar10b_sunrise.h5')#,custom_objects={'smoothL1':smoothL1})
print("Printing summary of the model:")
model.summary()


layer_name = 'dense'
lay = model.get_layer(layer_name)
print(lay.get_config())

layer_name = 'activation_4'
lay = model.get_layer(layer_name)
print(lay.get_config())

layer_name = 'dense_1'
lay = model.get_layer(layer_name)
print(lay.get_config())

layer_name = 'activation_5'
lay = model.get_layer(layer_name)
print(lay.get_config())






Loading the model:
Printing summary of the model:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
activa

In [ ]:
def check_pattern(layer_vals,suff,neuron_ids,neuron_sig,VAL = False,ALL = False):
   #layer = 'activation_3'  #layer name, get it from the model summary
   #neuron_ids =  [-1,71,-1,20,-1,1,-1,28,-1,79,-1,29,-1,39,-1,49,-1,42,-1,33,-1,114,-1,117,-1,88,-1,31,-1,36,-1,33,-1,17,-1,117,-1,1,-1,28,-1,97] # neuron ids in the pattern
   #neuron_sig =  ['>',0.435630053,'>',4.067234278,'<=',42.78802872,'<=',20.19964218,'<=',41.11421585,'<=',23.87646389,'<=',15.27161694,'<=',36.27468491,'<=',30.84732342,'<=',40.63782883,'<=',25.80421829,'<=',43.16035843,'<=',38.44324303,'<=',66.91875458,'<=',26.18983078,'<=',16.70914364,'<=',41.36343193,'<=',35.72474098,'<=',35.5690403,'<=',14.66370487,'<=',40.59473419189453]

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   found = True;

   oper = -1
   for ind in range(0,len(neuron_ids)):
     if (ind % 2 == 0):
       op = neuron_sig[ind]
       if (op == '<='):
         oper = 0
       else:
         oper = 1
     else:
       v = int(neuron_ids[ind])
       vsig = float(neuron_sig[ind])
       val = layer_vals[v]
       #print(v,vsig,val,oper)
       if (oper == 0):
         if (val > vsig):
           found = False
           break
       else:
         if (val <= vsig):
            found = False
            break
       oper = -1

   return found

In [ ]:
import numpy as np
import csv

num_classes = 10

with open('cifar_test', 'rb') as f1:
    testset = pickle.load(f1, encoding='bytes')

x_test = testset['x_test']
y_test = testset['y_test']

ORIG_X = x_test

print ('Number of samples', len(y_test))
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print("== DONE! ==\n== COLOR PREPROCESSING... ==")
# color preprocessing
x_test = x_test.astype('float32')
mean = [125.307, 122.95, 113.865]
std = [62.9932, 62.0887, 66.7048]
for i in range(3):
    x_test[:, :, :, i] = (x_test[:, :, :, i] - mean[i]) / std[i]

test_Y = (y_test).argmax(axis=1)
test_X = x_test

#a=[]
#path = 'test_Y.txt'
#inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
#for row in inputfile: # TRAVERSES ALL ROWS OF DATA
#    c=[]
#    for i in range(0,len(row)):
#        c.append(int(row[i]))
#    a.append(c)
#test_Y = np.array(a)
print("IDEAL LABELS:", test_Y.shape)

#a=[]
#path = 'test_X.txt'
#inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
#for row in inputfile: # TRAVERSES ALL ROWS OF DATA
#    c=[]
#    for i in range(0,len(row)):
#        c.append(int(row[i]))
#    c = np.reshape(c, (32,32,3))
#    a.append(c)
#test_X = np.array(a)
print("CLEAN DATA:", test_X.shape)

with open('sunrise_test_10000', 'rb') as f1:
    testset = pickle.load(f1, encoding='bytes')

x_test = testset['x_test']
y_test = testset['y_test']

POIS_ORIG_X = x_test

print ('Number of samples', len(y_test))
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print("== DONE! ==\n== COLOR PREPROCESSING... ==")
# color preprocessing
x_test = x_test.astype('float32')
mean = [125.307, 122.95, 113.865]
std = [62.9932, 62.0887, 66.7048]
for i in range(3):
    x_test[:, :, :, i] = (x_test[:, :, :, i] - mean[i]) / std[i]

poisoned_x = x_test
poisoned_Y = (y_test).argmax(axis=1)

#a=[]
#path = 'backdoor_test_X.txt'
#inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
#for row in inputfile: # TRAVERSES ALL ROWS OF DATA
#    c=[]
#    for i in range(0,len(row)):
#        c.append(int(row[i]))
#    c = np.reshape(c, (32,32,3))
#    a.append(c)
    #print(c)
    #print(c.shape)
#poisoned_x = np.array(a)
print("POISONED INPUTS:", poisoned_x.shape)
print("POSIONED_Y:", poisoned_Y.shape)

poisoned_labels = (model.predict(poisoned_x)).argmax(axis=1)
print("ACTUAL POISONED LABELS:", poisoned_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(poisoned_labels)):
  #print(poisoned_labels[indx])
  if (poisoned_labels[indx] == test_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",POISONED ACCURACY:", (PASS/(PASS+FAIL))*100.0)

test_labels = (model.predict(test_X)).argmax(axis=1)
print("ACTUAL TEST LABELS:", test_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels)):
  if (test_labels[indx] == test_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)




Number of samples 10000
== DONE! ==
== COLOR PREPROCESSING... ==
IDEAL LABELS: (10000,)
CLEAN DATA: (10000, 32, 32, 3)
Number of samples 10000
== DONE! ==
== COLOR PREPROCESSING... ==
POISONED INPUTS: (10000, 32, 32, 3)
POSIONED_Y: (10000,)
ACTUAL POISONED LABELS: (10000,)
PASS: 1012 ,FAIL: 8988 ,POISONED ACCURACY: 10.12
ACTUAL TEST LABELS: (10000,)
PASS: 8170 ,FAIL: 1830 ,TEST ACCURACY: 81.69999999999999


In [ ]:
from random import randint

gen_data = []
gen_labels = []
val_data = []
val_labels = []

gen_orig_data = []
val_orig_data = []
gen_pois_orig_data = []
val_pois_orig_data = []

indx_clean = []
len_test = len(test_X)
while (len(indx_clean) < 5000):
  value = randint(0,len_test)
  while (value in indx_clean):
    value = randint(0,len_test)
  indx_clean.append(value)


for indx in range(0,len(test_X)):
  if (indx in indx_clean):
    gen_data.append(test_X[indx])
    gen_labels.append(test_Y[indx])
    gen_orig_data.append(ORIG_X[indx])
    gen_pois_orig_data.append(-1)
  else:
 #   if (len(val_data) < 5000):
    val_data.append(test_X[indx])
    val_labels.append(test_Y[indx])
    val_orig_data.append(ORIG_X[indx])
    val_pois_orig_data.append(-1)


indx_poison = []
len_poison = len(poisoned_x)
while (len(indx_poison) < 5000):
  value = randint(0,len_poison)
  while (value in indx_poison):
    value = randint(0,len_poison)
  indx_poison.append(value)


for indx in range(0,len(poisoned_x)):
  if (indx in indx_poison):
    gen_data.append(poisoned_x[indx])
    gen_labels.append(test_Y[indx])
    gen_orig_data.append(POIS_ORIG_X[indx])
    gen_pois_orig_data.append(ORIG_X[indx])
  else:
 #   if (len(val_data) < 5000):
    val_data.append(poisoned_x[indx])
    val_labels.append(test_Y[indx])
    val_orig_data.append(POIS_ORIG_X[indx])
    val_pois_orig_data.append(ORIG_X[indx])

#for indx in range(0,int(len(poisoned_x)/2)):
#  gen_data.append(poisoned_x[indx])
#  gen_labels.append(test_Y[indx])
#  gen_orig_data.append(POIS_ORIG_X[indx])
#  gen_pois_orig_data.append(test_X[indx])

#for indx in range(int(len(poisoned_x)/2),len(poisoned_x) ):
#  val_data.append(poisoned_x[indx])
#  val_labels.append(test_Y[indx])
#  val_orig_data.append(POIS_ORIG_X[indx])
#  val_pois_orig_data.append(test_X[indx])



gen_data = np.asarray(gen_data)
gen_labels = np.asarray(gen_labels)
print("GENERATION DATASET:", gen_data.shape, gen_labels.shape)
preds = model.predict(gen_data)
actuals_gen = preds.argmax(axis=1)

val_data = np.asarray(val_data)
val_labels = np.asarray(val_labels)
print("VAL DATASET:", val_data.shape, val_labels.shape)
preds = model.predict(val_data)
actuals_val = preds.argmax(axis=1)




GENERATION DATASET: (9999, 32, 32, 3) (9999,)
VAL DATASET: (10001, 32, 32, 3) (10001,)


In [ ]:
#layer_preds
layer = model.get_layer('dense') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(gen_data)
fingerprint = ((outputs[0] > 0.0).astype('int'))
fingerprint1 = outputs[0]

PASS_LABEL_0 = 0
PASS_LABEL_1 = 0
PASS = 0
FAIL = 0
FAIL_P = 0
val_vec = []
val_vec_labels = []
val_vec_labels1 = []
print("VALS from layer:")
#file1 = open('TWITTER_VALS_DENSE.txt', 'w')
for i in range(0,len(fingerprint1)):
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    val_vec_labels.append(gen_labels[i])
    val_vec_labels1.append(gen_labels[i])
    val_vec.append(np.asarray((fingerprint1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_val[i] == 0):
        FAIL_P = FAIL_P+ 1
    if (actuals_gen[i] == 0):
      val_vec_labels.append(1000)
    else:
      val_vec_labels.append(0 - actuals_gen[i])
    if (gen_labels[i] == 0):
      val_vec_labels1.append(-1000)
    else:
      val_vec_labels1.append(0 - gen_labels[i])
    val_vec.append(np.asarray((fingerprint1[i]).flatten()))


#PASS = 0
#FAIL = 0
#act_vec = []
#act_vec_labels = []
#act_vec_labels1 = []
#gen_fail_labels = []
#FAIL_LABEL_0 = 0
#FAIL_LABEL_1 = 0
#print("ACTS from layer:")
#file1 = open('TWITTER_ACTS_DENSE.txt', 'w')
#FAIL_P = 0

#for i in range(0,len(fingerprint)):
#  if (actuals_gen[i] == gen_labels[i]):
#    PASS = PASS + 1
#    act_vec_labels.append(gen_labels[i])
#    act_vec.append(np.asarray((fingerprint[i]).flatten()))
#  else:
#    FAIL = FAIL + 1
#    if (actuals_val[i] == 0):
#      FAIL_P = FAIL_P+ 1
#    act_vec.append(np.asarray((fingerprint[i]).flatten()))
#    if (actuals_gen[i] == 0):
#      act_vec_labels.append(-1000)
    #else:
    #  act_vec_labels.append(0 - actuals_gen[i])
    #if (gen_labels[i] == 0):
    #   act_vec_labels.append(-1000)
    #else:
    #   act_vec_labels.append(0 - gen_labels[i])

print("GENERATION SET FAIL:", FAIL, ", FAIL POISONED:", FAIL_P,",ACCURACY:" , (PASS/(PASS+FAIL))*100.0)


PASS = 0
FAIL = 0
PASS_LABEL_0 = 0
PASS_LABEL_1 = 0
FAIL_P = 0
for i in range(0,len(val_data)):
  if (actuals_val[i] == val_labels[i]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
    if (actuals_val[i] == 0):
      FAIL_P = FAIL_P+ 1


print("VALIDATION SET FAIL:", FAIL, ", FAIL POISONED:", FAIL_P,",ACCURACY:" , (PASS/(PASS+FAIL))*100.0)


VALS from layer:
GENERATION SET FAIL: 5401 , FAIL POISONED: 4552 ,ACCURACY: 45.984598459845984
VALIDATION SET FAIL: 5417 , FAIL POISONED: 4625 ,ACCURACY: 45.83541645835417


In [ ]:
from sklearn import tree

start_time1 = time.time()

val_vec = np.asarray(val_vec)
val_vec_labels = np.asarray(val_vec_labels)
print(val_vec.shape, val_vec_labels.shape)

val_vec_labels1 = np.asarray(val_vec_labels1)
print(val_vec.shape, val_vec_labels1.shape)

## MISCLASSIFICATION PATTERN WITH ACTUAL LABELS
train_suffixes = val_vec
train_predictions = val_vec_labels
basic_estimator = tree.DecisionTreeClassifier()
basic_estimator.fit(val_vec, val_vec_labels)

## MISCLASSIFICATION PATTERN WITH IDEAL LABELS
train_suffixes1 = val_vec
train_predictions1 = val_vec_labels1
basic_estimator1 = tree.DecisionTreeClassifier()
basic_estimator1.fit(val_vec, val_vec_labels1)

PATTERN_TIME = time.time() - start_time1

(9999, 512) (9999,)
(9999, 512) (9999,)


In [ ]:
from tqdm import tqdm
import operator
import pandas as pd

def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(neuron_ids, neuron_sig,suffixes=train_suffixes):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

def is_consistent_cluster(cluster, predictions):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = predictions[cluster[0]]
  for i in cluster:
    if predictions[i] != pred:
      return False
  return True



def get_invariant_inp(estimator, ref_id, suffixes):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - inp: reference input, shape <784,>
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist_inp_images[ref_id]
  ref_suffix = suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,suffixes)
  imgs = []
  cnt = 0
  for indx1 in range(0,len(cluster)):
    img = mnist.train.images(cluster[indx1])
    fnd = 1
    for i in range(0,len(img)):
      if (ref_img[i] != img[i]):
        fnd = 0
        break
    if (fnd == 1):
        ref_id = cnt
    cnt = cnt + 1
    imgs.append(img)

  imgs_suffixes = fingerprint_signature(imgs,t_fc2)
  ref_suffix = imgs_suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,imgs_suffixes)

  return cluster, neuron_ids, neuron_sig

def get_invariant(estimator, ref_id):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into mnist.train.images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist.train.images[ref_id]
  ref_suffix = train_suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  neuron_sig = ref_suffix[neuron_ids]
  cluster = get_suffix_cluster(neuron_ids, neuron_sig)
  return cluster, neuron_ids, neuron_sig


def get_all_invariants(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]

  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
   # paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
   # paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    paths = [[[feature[node]] + p[0],['<='] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0],['>'] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print("Obtained all paths")
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants


def describe_cluster(cluster, neuron_ids, show_samples=False):
  neuron_sig = train_suffixes[cluster[0]][neuron_ids]
  print("Num neurons in invariant", len(neuron_ids))
  print("Neuron id and signature")

  for i in range(0,len(neuron_ids)):
    print("id:", neuron_ids[i], "sig:", neuron_sig[i])

  print("Cluster size: ", len(cluster))
  print("Num misclassified", len([i for i in cluster if is_misclassified(i)]))
  if show_samples:
    for i in range(10):
      images = []
      for j in range(10):
        if 10*i + j >= len(cluster):
          break
        images.append(mnist_to_pil_img(mnist.train.images[cluster[10*i+j]]))
      if len(images) > 0:
        show_img(combine(images))

def describe_invariants_all_labels_col(all_invariants,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  invs_sel_neus = []
  invs_sel_sig = []
  Minvs_sel_neus = []
  Minvs_sel_sig = []
  Minvs_sel_supp = []
  invs_sel_supp = []
  invs_sel_cl = []
  Minvs_sel_cl = []

  corr_invs_sel_labs = []
  corr_invs_sel_neus = []
  corr_invs_sel_sig = []
  corr_invs_sel_supp = []

  for cl, invs in all_invariants.items():

    for indx in range (0, len(invs)):
      inv = invs[indx]


     # print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])

    # MISCLASSIFICATION PATTTERN FOR IDEAL LABELs
      if (cl < 0):
          Minvs_sel_cl.append(cl)
          Minvs_sel_neus.append(inv[0])
          Minvs_sel_sig.append(inv[1])
          Minvs_sel_supp.append(inv[2])

      # MISCLASSIFICATION PATTTERNs FOR ACTUAL LABELS
      if (cl == 1000):
        invs_sel_neus.append(inv[0])
        invs_sel_sig.append(inv[1])
        invs_sel_supp.append(inv[2])
        invs_sel_cl.append(cl)


      if ((cl >= 0) and (cl not in corr_invs_sel_labs)):
          corr_invs_sel_labs.append(cl)
          corr_invs_sel_neus.append(inv[0])
          corr_invs_sel_sig.append(inv[1])
          corr_invs_sel_supp.append(inv[2])



  return (Minvs_sel_neus, Minvs_sel_sig, Minvs_sel_supp, Minvs_sel_cl, invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp)


def describe_invariants_all_labels(all_invariants,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  invs_sel_neus = []
  invs_sel_sig = []
  invs_sel_supp = []
  corr_invs_sel_labs = []
  corr_invs_sel_neus = []
  corr_invs_sel_sig = []
  corr_invs_sel_supp = []

 # print("PRINTING PURE RULES WITH SUPPORT MORE THAN 100 FOR EVERY LABEL:");
  supp = 0
  for cl, invs in all_invariants.items():

    for indx in range (0, len(invs)):
      inv = invs[indx]
      if (inv[2] <= 10):
        continue
      #print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])
      print("Class:", cl, "Support:",inv[2])
      if (cl == -1000):
        if (inv[2] > ((10/100) * FAIL_P)):
          invs_sel_neus.append(inv[0])
          invs_sel_sig.append(inv[1])
          invs_sel_supp.append(inv[2])
          supp = supp + inv[2]

      if ((cl >= 0) and (cl not in corr_invs_sel_labs)):
          corr_invs_sel_labs.append(cl)
          corr_invs_sel_neus.append(inv[0])
          corr_invs_sel_sig.append(inv[1])
          corr_invs_sel_supp.append(inv[2])

      continue

      cls = get_suffix_cluster(inv[0],inv[1],suffixes)

      neurons = inv[0]
      signature = inv[1]

      if (len(cls) <= 10):
         continue

     # fail_cnt = []
     # fail_label = []
     # pass_cnt = []
     # for ind in range(0,len(cls)):
     #   inp_indx = cls[ind]
     #   if (FP[inp_indx] == 0):
     #     fail_cnt.append(inp_indx)
     #     fail_label.append(IDEAL[inp_indx])
     #   else:
     #     pass_cnt.append(inp_indx)

      #fail_perc = (fail_cnt/len(cls))
      #if (inv[2] == 243):
      #print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2],",Pass:",[i for i in pass_cnt],",Mis:",[i for i in fail_cnt] , ",Label:",[i for i in fail_label]);

      print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])
      #if (indx > 0):
      #  break
     # for ind in range(0, len(cls)):
     #    print(cls[ind])

     # break
      if (COMMON == True):
          common_nodes(cls,suffixes)

      if (DEC_PREFX == True):
          decision_prefs(cls,suffixes)

  return (invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp)


def common_nodes(cls,suffixes):
    cnt = 0
    common = np.zeros(10,dtype=int)
    prev = np.zeros(10,dtype=int)

    for indx in range(0, len(cls)):
        i = cls[indx]
        cnt = cnt + 1
        for j in range(0,len(suffixes[i])):
          if (common[j] == -1):
             continue
          if ((indx != 0) and (suffixes[i][j] != prev[j])):
             common[j] = -1
          else:
             common[j] = suffixes[i][j]
          prev[j] = suffixes[i][j]


    print('COMMON NODES IN CLUSTER for CLASS:',cl,cnt)
    com = []
    for k in range(0,len(common)):
        if (common[k] != -1):
           com.append((k,common[k]))
    print(com)

    return

def decision_prefs(cls,suffixes):
    images = mnist.train.images
    imgsCom = []
    imgs = []
    for indx in range(0, len(cls)):
        print('IMG:')
        print(list(zip(images[cls[indx]])))
        imgs.append(images[cls[indx]])
        imgsCom.append(images[cls[indx]])

    dec_prefixes= fingerprint_signature(imgs,layer)
    prefixes = []
    for indx in range(0,len(dec_prefixes)):
       dec_pref = dec_prefixes[indx]

       match = 0
       for indx1 in range(0, len(prefixes)):
          match = 1
          for i in range(0,len(prefixes[indx1])):
             if (dec_pref[i] != prefixes[indx1][i]):
                match = 0
                break
          if (match == 1):
             break

       if (match == 0):
          prefixes.append(dec_pref)

    print('DECISION PREFIXES IN CLUSTER for CLASS:',cl,cnt)
    for k in range(0,len(prefixes)):
      print(prefixes[k])

    return


In [ ]:
#invariants = get_all_invariants(basic_estimator)
#(Ainvs_sel_neus, Ainvs_sel_sig, Ainvs_sel_supp, Acorr_invs_sel_labs, Acorr_invs_sel_neus, Acorr_invs_sel_sig, Acorr_invs_sel_supp) = describe_invariants_all_labels(invariants)
#print("ACTIVATION PATTERNS:")
#for indx in range(0, len(Ainvs_sel_neus)):
#  print(Ainvs_sel_neus[indx], Ainvs_sel_sig[indx], Ainvs_sel_supp[indx])
#for indx in range(0, len(Acorr_invs_sel_labs)):
#  print(Acorr_invs_sel_labs[indx], Acorr_invs_sel_neus[indx], Acorr_invs_sel_sig[indx], Acorr_invs_sel_supp[indx])


#invariants = get_all_invariants(basic_estimator1)
#(invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants)


invariants_ACT = get_all_invariants(basic_estimator)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels_col(invariants_ACT)


print("\nACTUAL LABEL based PATTERNS:\n")
for indx in range(0, len(invs_sel_neus)):
  print(invs_sel_cl[indx],invs_sel_neus[indx], invs_sel_sig[indx], invs_sel_supp[indx])

invariants_IDEAL = get_all_invariants(basic_estimator1)
(Minvs_sel_neus, Minvs_sel_sig, Minvs_sel_supp, Minvs_sel_cl, invs_sel_neus1, invs_sel_sig1, invs_sel_supp1, invs_sel_cl1, corr_invs_sel_labs1, corr_invs_sel_neus1, corr_invs_sel_sig1, corr_invs_sel_supp1) = describe_invariants_all_labels_col(invariants_IDEAL)

print("\nIDEAL LABEL based PATTERNS:\n")
for indx in range(0, len(Minvs_sel_neus)):
   print(Minvs_sel_cl[indx], Minvs_sel_neus[indx], Minvs_sel_sig[indx], Minvs_sel_supp[indx])


In [ ]:
# INVOKE FOR EVERY CORRECT LABEL
def corr_match_pattern(layer_vals,suff,neuron_ids,neuron_sig):
   max_match_cnt = -1
   for ix in range(0,len(neuron_ids)):
     match_cnt = 0
     oper = -1
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         # print(v,vsig,val,oper)
         if (oper == 0):
          if (val <= vsig):
            match_cnt = match_cnt + 1
         else:
          if (val > vsig):
            match_cnt = match_cnt + 1
         oper = -1

     if (match_cnt > max_match_cnt):
       max_match_cnt = match_cnt

   return max_match_cnt

def check_pattern_inter(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False


   found = True;
   oper = -1
   layer_vals = (layer_vals).flatten()

   for ix in range(0,len(neuron_ids)):
     found = True
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         #print(oper, v, vsig, val)
         if (oper == 0):
          if (val > vsig):
            found = False
          #  print('False')
            break
         else:
          if (val <= vsig):
            found = False
           # print('False')
            break
         oper = -1
     if (found == True):
       break

   if (found == False):
      return -1
   else:
     return ix

In [ ]:
REPAIR = False
VAL = True
ALL = False


In [ ]:
def check_pattern(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):
   #layer = 'activation_3'  #layer name, get it from the model summary
   #neuron_ids =  [-1,71,-1,20,-1,1,-1,28,-1,79,-1,29,-1,39,-1,49,-1,42,-1,33,-1,114,-1,117,-1,88,-1,31,-1,36,-1,33,-1,17,-1,117,-1,1,-1,28,-1,97] # neuron ids in the pattern
   #neuron_sig =  ['>',0.435630053,'>',4.067234278,'<=',42.78802872,'<=',20.19964218,'<=',41.11421585,'<=',23.87646389,'<=',15.27161694,'<=',36.27468491,'<=',30.84732342,'<=',40.63782883,'<=',25.80421829,'<=',43.16035843,'<=',38.44324303,'<=',66.91875458,'<=',26.18983078,'<=',16.70914364,'<=',41.36343193,'<=',35.72474098,'<=',35.5690403,'<=',14.66370487,'<=',40.59473419189453]

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   found = True;
   #Integer v = null;
	#			Double vsig = null;
	#			String op = null;
	#			int oper = -1;
   oper = -1
   for ind in range(0,len(neuron_ids)):
     if (ind % 2 == 0):
       op = neuron_sig[ind]
       if (op == '<='):
         oper = 0
       else:
         oper = 1
     else:
       v = neuron_ids[ind]
       vsig = neuron_sig[ind]
       val = layer_vals[v]
      # print(v,vsig,val,oper)
       if (oper == 0):
         if (val > vsig):
           found = False
           break
       else:
         if (val <= vsig):
            found = False
            break
       oper = -1

   return found

In [ ]:
#### GUESSING THE IDEAL LABELS BASED ON PATTERNS
neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

for indx in range (0, len(invs_sel_neus)):
       neu_ids = []
       for indx1 in range (0, len(invs_sel_neus[indx])):
          neu_ids.append(-1)
          neu_ids.append(invs_sel_neus[indx][indx1])
       neuron_ids.append(neu_ids)
       inc_neuron_ids.append(invs_sel_neus[indx])

for indx in range (0, len(invs_sel_sig)):
       neuron_sig.append(invs_sel_sig[indx])

Mneuron_ids =[]
Minc_neuron_ids=[]
Mneuron_sig = []
Mneuron_cl = []

for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_cl[iy]
          Minvs_sel_cl[iy] = Minvs_sel_cl[ix]
          Minvs_sel_cl[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      Mneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        Mneu_ids.append(-1)
        Mneu_ids.append(Minvs_sel_neus[indx][indx1])
      Mneuron_ids.append(Mneu_ids)
      Minc_neuron_ids.append(Minvs_sel_neus[indx])


for indx in range (0, len(Minvs_sel_sig)):
        Mneuron_sig.append(Minvs_sel_sig[indx])
        Mneuron_cl.append(Minvs_sel_cl[indx])

print("\n MIS-CLASS PATTERNS FOR IDEAL LABELS\n")
for indx in range (0, len(Minvs_sel_neus)):
  print(Minvs_sel_cl[indx],Minvs_sel_supp[indx],Minvs_sel_neus[indx],Minvs_sel_sig[indx])

import copy
x = copy.deepcopy(val_data)
cnt = 0
correct = 0

labels = actuals_val #actuals_gen
ideals = val_labels #gen_labels


true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

start_time = time.time()
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern

    image = x[inp_cnt]
    label = labels[inp_cnt]

    inp = image
    inc_layer = 'dense'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)



    layer_vals = inc_suff[0]
    suff = []

    image_array = np.zeros((1,32,32,3),dtype=float)
    image_array[0]=image

    img = np.zeros((32,32,3),dtype=float)
    for ix in range(0,32):
      for iy in range(0,32):
        for iz in range(0,3):
          img[ix][iy][iz] = (float(image[ix][iy][iz]))

    img_mod = np.expand_dims(img,axis=0)


    match = check_pattern_inter(layer_vals,suff,neuron_ids,neuron_sig,VAL,False)

    if (match >= 0):

        print("IDENTIFIED POISONED INPUT", inp_cnt,",", cnt, ",ACT:", labels[inp_cnt], ",IDEAL:", ideals[inp_cnt])
        cnt = cnt + 1

        if (labels[inp_cnt] != 0):
          false_positives = false_positives + 1
        else:
          true_positives = true_positives + 1

        match1 = check_pattern_inter(layer_vals,suff,Mneuron_ids,Mneuron_sig,VAL,ALL)

        if (match1 == -1):
          cnt_not_fnd = cnt_not_fnd + 1
          new_lbl = randint(1,9)
        else:
          new_lbl = -Mneuron_cl[match1]

        print("MATCH1,NEW LABEL:", match1,new_lbl)

        if (new_lbl == ideals[inp_cnt]): #and (labels[inp_cnt] != ideals[inp_cnt]) ):
           correct = correct + 1
           PASS = PASS + 1
        else:
           FAIL = FAIL + 1
           if (labels[inp_cnt] == ideals[inp_cnt]):
             broke = broke + 1

        if (labels[inp_cnt] == ideals[inp_cnt]):
          PASS_bef = PASS_bef + 1
        else:
          FAIL_bef = FAIL_bef + 1


    else:

      if (labels[inp_cnt] != 0):
          true_negatives = true_negatives + 1
      else:
          false_negatives = false_negatives + 1

      if (labels[inp_cnt] == ideals[inp_cnt]):
        PASS_bef = PASS_bef + 1
        PASS = PASS + 1
      else:
        FAIL_bef = FAIL_bef + 1
        FAIL = FAIL + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")
print('TOTAL IDENTIFIED AS POISONED :',cnt)


print('TP:', true_positives, ',FP:', false_positives, ",TN:", true_negatives, ",FN:", false_negatives)
PREC = ((true_positives)/ (true_positives + false_positives))
RECALL = ((true_positives)/ (true_positives + false_negatives))
print("PRECISION:" , PREC * 100.0, ",RECALL:", RECALL * 100.0)

print('TOTAL GUESSED BASED ON PATTERNs :',(cnt - cnt_not_fnd))
print('CORRECT:', correct)
print('BROKE PREVIOUSLY CORRECTLY CLASSIFIED:', broke)

rep_rate = ((correct/true_positives) * 100.0)
print("REPAIR RATE:", rep_rate)
rep_rate = ((correct/(true_positives + false_negatives)) * 100.0)
print("FULL REPAIR RATE:", rep_rate)


#print("RUNTIME TIME CONSUMED:" , time.time() - start_time)
print("OVERALL ACCURACY without monitor:", (PASS_bef/(PASS_bef+FAIL_bef)) * 100 )
print("OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

print("OFFLINE TIME CONSUMED:" ,PATTERN_TIME)
print("RUNTIME TIME CONSUMED:" , time.time() - start_time)


Streaming output truncated to the last 5000 lines.
IDENTIFIED POISONED INPUT 7149 , 2022 ,ACT: 0 ,IDEAL: 9
MATCH1,NEW LABEL: 1301 4
IDENTIFIED POISONED INPUT 7151 , 2023 ,ACT: 0 ,IDEAL: 9
MATCH1,NEW LABEL: 550 2
IDENTIFIED POISONED INPUT 7152 , 2024 ,ACT: 0 ,IDEAL: 1
MATCH1,NEW LABEL: 528 7
IDENTIFIED POISONED INPUT 7153 , 2025 ,ACT: 0 ,IDEAL: 6
MATCH1,NEW LABEL: 510 2
IDENTIFIED POISONED INPUT 7154 , 2026 ,ACT: 0 ,IDEAL: 5
MATCH1,NEW LABEL: 372 7
IDENTIFIED POISONED INPUT 7155 , 2027 ,ACT: 0 ,IDEAL: 4
MATCH1,NEW LABEL: 133 4
IDENTIFIED POISONED INPUT 7156 , 2028 ,ACT: 0 ,IDEAL: 1
MATCH1,NEW LABEL: 1287 4
IDENTIFIED POISONED INPUT 7157 , 2029 ,ACT: 0 ,IDEAL: 8
MATCH1,NEW LABEL: -1 2
IDENTIFIED POISONED INPUT 7158 , 2030 ,ACT: 0 ,IDEAL: 9
MATCH1,NEW LABEL: 110 3
IDENTIFIED POISONED INPUT 7159 , 2031 ,ACT: 0 ,IDEAL: 8
MATCH1,NEW LABEL: 38 5
IDENTIFIED POISONED INPUT 7160 , 2032 ,ACT: 0 ,IDEAL: 8
MATCH1,NEW LABEL: -1 8
IDENTIFIED POISONED INPUT 7161 , 2033 ,ACT: 0 ,IDEAL: 6
MATCH1,NEW LAB